In [ ]:
''' EE954 : Deep Learning Fundamentals
    Group Number: 12
    Team Members: 
    - Lokesh  (Roll no: 241562482)
    - Akshay (Roll No:)'''

### Assignment 1

' EE954 : Deep Learning Fundamentals\n    Group Number: 12\n    Team Members: \n    - Lokesh  (Roll no: )\n    - Akshay (Roll No:)'